In [ ]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
import spaghetti
import os
import glob
import networkx as nx
from matplotlib import pyplot as plt
from pyproj import Proj
import smopy
import collections
import numpy as np
import math
import esda
from haversine import haversine
import warnings
from shapely.geometry import Point, LineString
from haversine import haversine

### Select city area

In [ ]:
location_researched = "Centre of Leeds, United Kingdom"
# buffer_dist is to specifies the distance in meters to buffer the geocoded point. 
leeds_centre = ox.geocode_to_gdf(location_researched, buffer_dist=1000)
leeds_polygon = leeds_centre['geometry'][0]
leeds_polygon

In [ ]:
# get the position of the Leeds City Centre which we choose to be the original area.
# leeds_centre.lat.values,leeds_centre.lon.values
# The longitudes and latitudes of the central points are：
print("Latitude of the central points is:",leeds_centre.lat.values)
print("longitude of the central points is:",leeds_centre.lon.values)

In [ ]:
# The years we used to generate the network are from 2014-2019
year_used = ['2014','2015','2016','2017','2018','2019']# '2014','2015','2016',
leeds_stored = []
for i in range(len(year_used)):
    # ISO-8859-1 encoding is a single-byte encoding, backwards compatible with ASCII, 
    # and is the encoding standard used in many European countries.
    leeds_flag = pd.read_csv('./'+year_used[i] +'.csv',encoding='ISO-8859-1')
    leeds_stored.append(leeds_flag)
leeds_df = pd.concat([k for k in leeds_stored], axis=0, ignore_index=True)
leeds_df.head(6)

In [ ]:
leeds_df.dropna(subset=['Grid Ref: Easting', 'Grid Ref: Northing'], inplace=True)
leeds_df

In [ ]:
# geometry=gpd.points_from_xy(leeds_df["Grid Ref: Easting"], leeds_df["Grid Ref: Northing"],crs="EPSG:27700")
leeds_gdf = gpd.GeoDataFrame(leeds_df, geometry=gpd.points_from_xy(leeds_df["Grid Ref: Easting"], leeds_df["Grid Ref: Northing"],crs="EPSG:27700"))
leeds_new_gdf=leeds_gdf.to_crs(epsg=4326).drop(['Grid Ref: Northing','Grid Ref: Easting'],axis=1)
leeds_new_gdf

In [ ]:
accident_nodes = leeds_new_gdf.geometry
# Select the nodes that are in the pre-determined range
leeds_accident = leeds_new_gdf[accident_nodes.within(leeds_polygon)]
len(leeds_accident)

In [ ]:
# in order to show the result of the accident points with the defined area, I draw them together.
tt = leeds_accident.geometry
box_stations = (tt.y.min(),tt.x.min(),
       tt.y.max(), tt.x.max())
map = smopy.Map(box_stations, z=100) # z is the zoom level, here we set it as 10, you may want to try another value.
# map.show_ipython()
p1 = pd.Series(leeds_polygon.exterior.coords.xy[1].tolist())
p2 = pd.Series(leeds_polygon.exterior.coords.xy[0].tolist())
x1, y1 = map.to_pixels(tt.y, tt.x)
x2, y2 = map.to_pixels(p1,p2)
ax = map.show_mpl(figsize=(12, 10))
ax.plot(x1, y1, 'o',color='crimson',ms=4, mew=1) # 'or' means red dots/circles
ax.plot(x2, y2, color='darkblue', linestyle='solid',linewidth=4, markersize=20)

In [ ]:
# Download the road network data
leeds_graph = ox.graph_from_polygon(leeds_polygon, network_type='drive')
nx.check_planarity(leeds_graph)

In [ ]:
num_nodes = leeds_graph.number_of_nodes()
num_edges = leeds_graph.number_of_edges()
print("The number of nodes in this road network is: ",num_nodes)
print("The number of edges in this road network is: ",num_edges)

In [ ]:
leeds_projection = ox.project_gdf(leeds_centre).unary_union.area
ox.basic_stats(leeds_graph, area=leeds_projection)

In [ ]:
all_shortest_paths = dict(nx.all_pairs_dijkstra_path_length(G, weight='length'))

# Cast the keys to integers
all_shortest_paths = {int(k):v for k,v in all_shortest_paths.items()}

# Find the longest of these shortest paths
longest_shortest_path = max([max(path_lengths.values()) for path_lengths in all_shortest_paths.values()])

print("The spatial diameter is same as the longest shortest path in the network, which is: {:.2f} meters".format(longest_shortest_path))

In [ ]:
nx.check_planarity(G)

In [ ]:
x_values = nx.get_node_attributes(leeds_graph, 'x')
y_values = nx.get_node_attributes(leeds_graph, 'y')

# We will work with the edges and add the missing geometries (lines denoting the roads between points)
graph_with_geometries = list(leeds_graph.edges(data=True))

# Iterate through the edges and, where missing, add a geometry attribute with the line between start and end nodes
for e in graph_with_geometries:
    if not 'geometry' in e[2]:
        e[2]['geometry'] = LineString([
            Point(x_values[e[0]], y_values[e[0]]),
            Point(x_values[e[1]], y_values[e[1]])])

graph_with_geometries[0:4]

### Task2

In [ ]:
# We will now drop the start and end nodes, as we will construct a new Spaghetti network based on the geometries of the roads
road_lines = [x[2] for x in graph_with_geometries]
# From this, we can construct a GeoDataFrame
roads_geodataframe = gpd.GeoDataFrame(pd.DataFrame(road_lines))
roads_geodataframe

In [ ]:
# From the GeoDataFrame, we can construct a network in Spaghetti from which to do point analysis
leeds_points_graph = spaghetti.Network(in_data=roads_geodataframe)

In [ ]:
# Check what this new network looks like by getting DataFrames for the edges and plotting them

nodes_df, edges_df = spaghetti.element_as_gdf(leeds_points_graph, vertices=True, arcs=True)
base_network = edges_df.plot(color="k", zorder=0, figsize=(15, 15))
nodes_df.plot(ax=base_network, color="r", zorder=2)

In [ ]:
# We will now snap the leeds located accidents we extracted earlier, i.e. position them at the closest point on the closest road
leeds_points_graph.snapobservations(leeds_accident, 'accidents')

# We can see the difference between the original accident coordinates and their position when snapped to the road network
print("observation 1\ntrue coords:\t%s\nsnapped coords:\t%s" % (
    leeds_points_graph.pointpatterns["accidents"].points[0]["coordinates"],
    leeds_points_graph.pointpatterns["accidents"].snapped_coordinates[0]
))

In [ ]:
# Show the network
base_network = edges_df.plot(color="k", zorder=0, figsize =(12, 12))
# Get a GeoDataFrame of the snapped accident locations to plot on the network image
snapped_accidents=spaghetti.element_as_gdf(
    leeds_points_graph, pp_name='accidents', snapped=True)

# Plot these on the road network
snapped_accidents.plot(
    color="r", marker="x",
    markersize=50, zorder=1, ax=base_network)

plt.show()

In [ ]:
# Show the network
base_network = edges_df.plot(color="k", zorder=0, figsize =(12, 12))
# Get a GeoDataFrame of the non-snapped (real) crime locations to plot on the net
observed_accidents=spaghetti.element_as_gdf(
    leeds_points_graph, pp_name='accidents', snapped=False)

# Plot these on the road network
observed_accidents.plot(
    color="r", marker="x",
    markersize=50, zorder=1, ax=base_network)

plt.savefig('accidents-observations')

In [ ]:
kres = leeds_points_graph.GlobalAutoK(
    leeds_points_graph.pointpatterns["accidents"],
    nsteps=100, permutations=100
)

In [ ]:
kres.lam
kres.xaxis
kres.observed
kres.upperenvelope
kres.lowerenvelope
kres.sim

print(f"Density of points in the network (lambda): {kres.lam}")

In [ ]:
print(f"Distances at which density is measured:\n{kres.xaxis}")

In [ ]:
fig, ax = plt.subplots()

ax.plot(kres.xaxis, kres.observed, "b-", label="Observed")
ax.plot(kres.xaxis, kres.upperenvelope, "r--", label="Upper")
ax.plot(kres.xaxis, kres.lowerenvelope, "k--", label="Lower")

ax.legend(loc="best", fontsize="x-large")
ax.set_xlabel("Distance $(r)$")
ax.set_ylabel("$K(r)$")

fig.tight_layout()

In [ ]:
# Get snapped point pattern 
pointpat = leeds_points_graph.pointpatterns['accidents']
# Get count of points per network edge: a dictionary from each edge to the crime count on that edge
counts = leeds_points_graph.count_per_link(pointpat.obs_to_arc, graph=False)

In [ ]:
counts

In [ ]:
moran, yaxis_moran = leeds_points_graph.Moran('accidents',graph=True)
moran.I

In [ ]:
moran, yaxis_moran = leeds_points_graph.Moran("accidents")
moran.I

In [ ]:
# Get the weights matrix for edges in the graph (just the adjacency matrix with 1 where edges connect at a node, 0 otherwise)
weights = leeds_points_graph.w_network
# Get the edges included in the weights matrix: an enumerator for a list of edges
edges = weights.neighbors.keys()
# Construct an array of the counts values per edge in the same order as
# the weights matrix, with 0.0 where no counts recorded
values = [counts[edge] if edge in counts.keys () else 0. \
    for index, edge in enumerate(edges)]


In [ ]:
moran = esda.moran.Moran(values, weights)
moran.I

In [ ]:
moran.p_sim

In [ ]:
import seaborn as sns
from splot.esda import moran_scatterplot, lisa_cluster, plot_moran
figsize = (20,10)
plot_moran(moran, zstandard=True, scatter_kwds=None, figsize=figsize)

In [ ]:
sns.kdeplot(moran.sim, shade=True)
plt.vlines(moran.I, 0, 1, color='r')
plt.vlines(moran.EI, 0,1)
plt.xlabel("Moran's I")

In [ ]:
snapped_accidents

In [ ]:
roads_geodataframe.geometry

In [ ]:
X = snapped_accidents.geometry.x
Y = snapped_accidents.geometry.y
dist_point_street = ox.distance.nearest_edges(leeds_graph, X, Y, interpolate=None, return_dist=False)

In [ ]:
dist_point_street

In [ ]:
num_accidents = len(dist_point_street)

In [ ]:
dist = [haversine((Y[i],X[i]), (leeds_graph.nodes()[dist_point_street[i][0]]['y'],leeds_graph.nodes()[dist_point_street[i][0]]['x']), unit='m') 
        if haversine((Y[i],X[i]), (leeds_graph.nodes()[dist_point_street[i][0]]['y'],leeds_graph.nodes()[dist_point_street[i][0]]['x']), unit='m') 
           <= haversine((Y[i],X[i]), (leeds_graph.nodes()[dist_point_street[i][1]]['y'],leeds_graph.nodes()[dist_point_street[i][1]]['x']), unit='m')
        else haversine((Y[i],X[i]), (leeds_graph.nodes()[dist_point_street[i][1]]['y'],leeds_graph.nodes()[dist_point_street[i][1]]['x']), unit='m') 
        for i in range(len(dist_point_street))]
percentage = [dist[i] / haversine((leeds_graph.nodes()[dist_point_street[i][1]]['y'],leeds_graph.nodes()[dist_point_street[i][1]]['x']), 
                              (leeds_graph.nodes()[dist_point_street[i][0]]['y'],leeds_graph.nodes()[dist_point_street[i][0]]['x']), unit='m')
            if haversine((Y[i],X[i]), (leeds_graph.nodes()[dist_point_street[i][0]]['y'],leeds_graph.nodes()[dist_point_street[i][0]]['x']), unit='m') 
               <= haversine((Y[i],X[i]), (leeds_graph.nodes()[dist_point_street[i][1]]['y'],leeds_graph.nodes()[dist_point_street[i][1]]['x']), unit='m')
            else dist[i] / haversine((leeds_graph.nodes()[dist_point_street[i][1]]['y'],leeds_graph.nodes()[dist_point_street[i][1]]['x']), 
                                     (leeds_graph.nodes()[dist_point_street[i][0]]['y'],leeds_graph.nodes()[dist_point_street[i][0]]['x']), unit='m')
            for i in range(len(dist_point_street))]

In [ ]:
dist

In [ ]:
ave_distance = sum(dist)/num_accidents
ave_percentage = sum(percentage)/num_accidents
print('The average distance between accident point and the intersection is: ',round(ave_distance,3),' meters') 
print('The average position percentage to intersection is: ',round(ave_percentage*100,3),' %')

### Task3

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
import distance

In [ ]:
PLACE_OF_INTEREST = "Leeds, United Kingdom"
# Get the network of streets in Soho
leeds_graph = ox.graph_from_place(PLACE_OF_INTEREST,network_type='drive')

In [ ]:
ox.plot_graph(leeds_graph)

In [ ]:
# We can see that the nodes are specified by x, y, coordinates
list(leeds_graph.nodes(data=True))[0:10]

In [ ]:
# And the edges are tuples of start node, end node and a dictionary with attributes including geometry
list(leeds_graph.edges(data=True))[0:3]

### Even distribution

In [ ]:
import numpy as np
from sklearn.cluster import KMeans

NUMBER_OF_SEEDS = 10
nodes = list(leeds_graph.nodes)

# Extract x,y coordinates of each node
coordinates = [(leeds_graph.nodes[n]['x'], leeds_graph.nodes[n]['y']) for n in nodes]

# Use k-means++ to get initial seed locations
kmeans = KMeans(n_clusters=NUMBER_OF_SEEDS, init='k-means++').fit(coordinates)
seeds = kmeans.cluster_centers_.tolist()
all_nodes = list(leeds_graph.nodes)

In [ ]:
seeds = [ox.distance.nearest_nodes(leeds_graph, longitude, latidude) for longitude, latidude in seeds]

In [ ]:
distances = {seed: nx.single_source_dijkstra_path_length(
    leeds_graph, seed, weight='length') for seed in seeds}

In [ ]:
def nearest_from_list(node_distances):
    return sorted(node_distances, key=lambda node_length: node_length[1])[0] \
         if len(node_distances) > 0 else None

In [ ]:
def nearest_seed(node):
    seed_distances = [(seed, distances[seed][node]) \
        for seed in seeds if node in distances[seed]]
    return nearest_from_list(seed_distances)

In [ ]:
def nearest_for_edge(edge):
    nearest_to_ends_all = [nearest_seed(edge[0]), nearest_seed(edge[1])]
    nearest_to_ends = [distance for distance in nearest_to_ends_all if distance]
    return nearest_from_list(nearest_to_ends)
colours = ox.plot.get_colors(NUMBER_OF_SEEDS)

In [ ]:
def colour_for_seed_distance(seed):
    return colours[seeds.index(seed[0])]

In [ ]:
edge_nearest_seeds = [nearest_for_edge(edge) for edge in leeds_graph.edges]
# Note that edges not connected to a seed shown in black, so invisible on black background
edge_colours = [colour_for_seed_distance(seed) if seed else 'k' for seed in edge_nearest_seeds]
# For the road network nodes, we want the seeds to be coloured red and the non-seed nodes to be coloured white.
node_colours = ['r' if node in seeds else 'none' for node in all_nodes]

In [ ]:
ox.plot.plot_graph(leeds_graph, edge_color = edge_colours, node_color = node_colours, bgcolor = 'k', save = True, filepath = 'nvd.png')

In [ ]:
seed_subgraphs = {}
for seed in seeds:
    seed_nodes = [node for node in list(leeds_graph.nodes()) if nearest_seed(node)!=None and nearest_seed(node)[0] == seed]
    seed_edges = [edge for edge in list(leeds_graph.edges()) if nearest_for_edge(edge)!=None and nearest_for_edge(edge)[0] == seed]
    seed_subgraphs[seed] = leeds_graph.subgraph(seed_nodes + [n for e in seed_edges for n in e])

In [ ]:
seed_subgraphs

In [ ]:
for seed, subgraph in seed_subgraphs.items():
    print(f"Seed {seed}:")
    print(f"Number of nodes: {subgraph.number_of_nodes()}")
    print(f"Number of edges: {subgraph.number_of_edges()}")

In [ ]:
ox.plot.plot_graph(seed_subgraphs[393344811])

In [ ]:
ox.plot.plot_graph(seed_subgraphs[384962202])

In [ ]:
G = seed_subgraphs[1955838372]
H = nx.Graph(G)  # 将多重图 G 转换成普通图 H
cycles = nx.cycle_basis(H) 
for cycle in cycles:
    subgraph = H.subgraph(cycle)
    length = sum([subgraph[u][v]['length'] for u, v in subgraph.edges()])
    print(f"Cycle: {cycle}, Length: {length}")

In [ ]:
for cycle in cycles:
    temp_subgraph = H.subgraph(cycle)
    length = sum([temp_subgraph[u][v]['length'] for u, v in temp_subgraph.edges()])
    if length<=44000 and length>=40000:
        # print(n)
        print("The length of the cycle is: ",length)
        break
print(temp_subgraph)

### seed_subgraphs[393344811]

In [ ]:
G = seed_subgraphs[393344811]
H = nx.Graph(G)  # 将多重图 G 转换成普通图 H
cycles = nx.cycle_basis(H) 
for cycle in cycles:
    subgraph = H.subgraph(cycle)
    length = sum([subgraph[u][v]['length'] for u, v in subgraph.edges()])
    print(f"Cycle: {cycle}, Length: {length}")

In [ ]:
for cycle in cycles:
    temp_subgraph = H.subgraph(cycle)
    length = sum([temp_subgraph[u][v]['length'] for u, v in temp_subgraph.edges()])
    if length<=42500 and length>=41500:
        # print(n)
        print("The length of the cycle is: ",length)
        break
print(temp_subgraph)


In [ ]:
fig, ax = plt.subplots(figsize=(250,250))
pos = nx.spring_layout(temp_subgraph, seed=42)
# nx.draw_networkx_edges(temp_subgraph, pos=pos, edge_color='r')

nx.draw_networkx_nodes(temp_subgraph, pos=pos, node_color='blue', node_size=500)
nx.draw_networkx_edges(temp_subgraph, pos=pos, edge_color='red', width=5)

plt.title(f"Cycle: {cycle}, Length: {length}")
plt.axis('off')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(350, 350))

# draw the entire graph H
pos = nx.spring_layout(H, seed=42)
nx.draw_networkx_nodes(H, pos=pos, node_color='blue', node_size=500)
nx.draw_networkx_edges(H, pos=pos, edge_color='gray', width=5)

# draw the subgraph temp_subgraph on top of the entire graph
nx.draw_networkx_nodes(temp_subgraph, pos=pos, node_color='red', node_size=500)
nx.draw_networkx_edges(temp_subgraph, pos=pos, edge_color='red', width=10)

plt.title(f"Cycle: {cycle}, Length: {length}")
plt.axis('off')
plt.show()

### seed_subgraphs[381915141]

In [ ]:
G = seed_subgraphs[381915141]
H = nx.Graph(G)  # 将多重图 G 转换成普通图 H
cycles = nx.cycle_basis(H) 

In [ ]:
for cycle in cycles:
    subgraph = H.subgraph(cycle)
    length = sum([subgraph[u][v]['length'] for u, v in subgraph.edges()])
    print(f"Cycle: {cycle}, Length: {length}")

In [ ]:
n=0
for cycle in cycles:
    n=n+1
    temp_subgraph = H.subgraph(cycle)
    length = sum([temp_subgraph[u][v]['length'] for u, v in temp_subgraph.edges()])
    if length<=42500 and length>=41500:
        print("The length of the cycle is: ",length)
        break
    else: 
        temp_subgraph == None
        length == 0
        
print(temp_subgraph)
print(length)

In [ ]:
fig, ax = plt.subplots(figsize=(250,250))
pos = nx.spring_layout(temp_subgraph, seed=42)
# nx.draw_networkx_edges(temp_subgraph, pos=pos, edge_color='r')

nx.draw_networkx_nodes(temp_subgraph, pos=pos, node_color='blue', node_size=500)
nx.draw_networkx_edges(temp_subgraph, pos=pos, edge_color='red', width=5)

plt.title(f"Cycle: {cycle}, Length: {length}")
plt.axis('off')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(350, 350))

# draw the entire graph H
pos = nx.spring_layout(H, seed=42)
nx.draw_networkx_nodes(H, pos=pos, node_color='blue', node_size=500)
nx.draw_networkx_edges(H, pos=pos, edge_color='gray', width=5)

# draw the subgraph temp_subgraph on top of the entire graph
nx.draw_networkx_nodes(temp_subgraph, pos=pos, node_color='red', node_size=500)
nx.draw_networkx_edges(temp_subgraph, pos=pos, edge_color='red', width=10)

plt.title(f"Cycle: {cycle}, Length: {length}")
plt.axis('off')
plt.show()

### Question 5

In [ ]:
import numpy as np
from sklearn.cluster import KMeans

NUMBER_OF_SEEDS = 4
nodes = list(leeds_graph.nodes)

# Set the random seed
np.random.seed(76)

# Extract x,y coordinates of each node
coordinates = [(leeds_graph.nodes[n]['x'], leeds_graph.nodes[n]['y']) for n in nodes]

# Use k-means++ to get initial seed locations
kmeans = KMeans(n_clusters=NUMBER_OF_SEEDS, init='k-means++').fit(coordinates)
seeds = kmeans.cluster_centers_.tolist()
all_nodes = list(leeds_graph.nodes)

In [ ]:
seeds = [ox.distance.nearest_nodes(leeds_graph, longitude, latidude) for longitude, latidude in seeds]

In [ ]:
distances = {seed: nx.single_source_dijkstra_path_length(
    leeds_graph, seed, weight='length') for seed in seeds}

In [ ]:
def nearest_from_list(node_distances):
    return sorted(node_distances, key=lambda node_length: node_length[1])[0] \
         if len(node_distances) > 0 else None
def nearest_seed(node):
    seed_distances = [(seed, distances[seed][node]) \
        for seed in seeds if node in distances[seed]]
    return nearest_from_list(seed_distances)

def nearest_for_edge(edge):
    nearest_to_ends_all = [nearest_seed(edge[0]), nearest_seed(edge[1])]
    nearest_to_ends = [distance for distance in nearest_to_ends_all if distance]
    return nearest_from_list(nearest_to_ends)
colours = ox.plot.get_colors(NUMBER_OF_SEEDS)

def colour_for_seed_distance(seed):
    return colours[seeds.index(seed[0])]

In [ ]:
edge_nearest_seeds = [nearest_for_edge(edge) for edge in leeds_graph.edges]
# Note that edges not connected to a seed shown in black, so invisible on black background
edge_colours = [colour_for_seed_distance(seed) if seed else 'k' for seed in edge_nearest_seeds]
# For the road network nodes, we want the seeds to be coloured red and the non-seed nodes to be coloured white.
node_colours = ['r' if node in seeds else 'none' for node in all_nodes]

In [ ]:
ox.plot.plot_graph(leeds_graph, edge_color = edge_colours, node_color = node_colours, bgcolor = 'k', save = True, filepath = 'nvd.png')

In [ ]:
seed_subgraphs = {}
for seed in seeds:
    seed_nodes = [node for node in list(leeds_graph.nodes()) if nearest_seed(node)!=None and nearest_seed(node)[0] == seed]
    seed_edges = [edge for edge in list(leeds_graph.edges()) if nearest_for_edge(edge)!=None and nearest_for_edge(edge)[0] == seed]
    seed_subgraphs[seed] = leeds_graph.subgraph(seed_nodes + [n for e in seed_edges for n in e])

In [ ]:
seed_subgraphs

In [ ]:
for seed, subgraph in seed_subgraphs.items():
    print(f"Seed {seed}:")
    print(f"Number of nodes: {subgraph.number_of_nodes()}")
    print(f"Number of edges: {subgraph.number_of_edges()}")

In [ ]:
G = seed_subgraphs[1984729084]
H = nx.Graph(G)  # 将多重图 G 转换成普通图 H
cycles = nx.cycle_basis(H) 
for cycle in cycles:
    subgraph = H.subgraph(cycle)
    length = sum([subgraph[u][v]['length'] for u, v in subgraph.edges()])
    print(f"Cycle: {cycle}, Length: {length}")

In [ ]:
for cycle in cycles:
    temp_subgraph = H.subgraph(cycle)
    length = sum([temp_subgraph[u][v]['length'] for u, v in temp_subgraph.edges()])
    if length<=42500 and length>=41500:
        # print(n)
        print("The length of the cycle is: ",length)
        break
print(temp_subgraph)